# 🧠 Hidden Markov Model (HMM) Analysis for Sleep EEG Signals

## 📋 Project Overview

This notebook implements a **GPU-accelerated Hidden Markov Model (HMM)** pipeline for EEG sleep signal analysis. The HMM approach models sleep as a sequence of hidden states (sleep stages) that generate observable EEG patterns, making it particularly suitable for:

- **Sleep stage classification** using temporal dependencies
- **State transition analysis** between different sleep phases  
- **Unsupervised clustering** of sleep patterns
- **GPU acceleration** for large-scale EEG datasets

## 🎯 Research Objectives

1. **Signal Preprocessing**: Clean and standardize EEG signals from EDF files
2. **HMM Implementation**: Custom GPU-accelerated HMM with TensorFlow
3. **State Classification**: Identify hidden sleep states from EEG patterns
4. **Performance Analysis**: Compare CPU vs GPU computational efficiency
5. **Clinical Application**: Apply to real sleep study data for stage detection

---

### 🏥 Clinical Context
Hidden Markov Models are particularly effective for sleep analysis because:
- Sleep stages follow **sequential patterns** (temporal dependencies)
- **State transitions** are probabilistic and clinically meaningful
- **Observable features** (EEG) are generated by hidden states (sleep stages)
- **Unsupervised approach** doesn't require labeled sleep staging data

---

### 📊 Expected Outcomes
- Automated sleep stage classification
- State transition probability matrices
- Performance comparison between CPU/GPU implementations
- Visualization of detected sleep architecture

In [ ]:
# ==============================================================================
# LIBRARY IMPORTS AND GPU CONFIGURATION
# ==============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne
from scipy.signal import detrend
from hmmlearn import hmm
import time
import warnings
warnings.filterwarnings('ignore')

# GPU acceleration libraries
import tensorflow as tf
import os

# Set TensorFlow logging level
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def check_gpu_availability():
    """Check and display GPU availability for acceleration."""
    print("🖥️  COMPUTING ENVIRONMENT STATUS")
    print("=" * 40)
    
    # Check TensorFlow devices
    physical_devices = tf.config.list_physical_devices()
    gpu_devices = tf.config.list_physical_devices('GPU')
    
    print(f"Available devices: {len(physical_devices)}")
    for device in physical_devices:
        print(f"  • {device}")
    
    if gpu_devices:
        print(f"\n✅ GPU Acceleration: Available ({len(gpu_devices)} GPU(s))")
        for i, gpu in enumerate(gpu_devices):
            print(f"  • GPU {i}: {gpu.name}")
        return True
    else:
        print("\n⚠️  GPU Acceleration: Not available - using CPU")
        return False

# Check GPU availability
gpu_available = check_gpu_availability()

2025-06-04 15:30:36.962521: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow sees the following devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Configure GPU memory management to prevent memory errors
def configure_gpu_memory():
    """Configure GPU memory growth to prevent allocation errors."""
    
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Enable memory growth for all GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print("✅ GPU memory growth configured successfully")
            
            # Set memory limit if needed (optional)
            # tf.config.experimental.set_memory_limit(gpus[0], 4096)  # 4GB limit
            
        except RuntimeError as e:
            print(f"❌ GPU configuration error: {e}")
            print("Continuing with default GPU settings...")
    else:
        print("No GPU devices found - skipping GPU configuration")

# Configure GPU memory
configure_gpu_memory()

Memory growth set to True


## 📁 Step 1: EEG Data Loading and Validation

Load EEG signals from EDF files with comprehensive validation and error handling.

In [ ]:
def load_eeg_data(file_path, channel_selection='auto'):
    """
    Load EEG signal from EDF file with comprehensive validation.
    
    Args:
        file_path (str): Path to EDF file
        channel_selection (str or list): Channel selection strategy
            - 'auto': Automatically select best EEG channel
            - 'first': Use first available channel
            - list: Specific channel names to load
    
    Returns:
        tuple: (eeg_data, sampling_freq, channel_info, metadata)
    """
    
    try:
        print(f"📁 Loading EEG data from: {file_path}")
        
        # Load EDF file with MNE
        raw = mne.io.read_raw_edf(file_path, preload=True, verbose=False)
        
        # Get sampling frequency
        sfreq = raw.info['sfreq']
        
        # Get channel information
        channel_names = raw.ch_names
        n_channels = len(channel_names)
        
        print(f"  • Sampling frequency: {sfreq} Hz")
        print(f"  • Number of channels: {n_channels}")
        print(f"  • Recording duration: {raw.times[-1]:.1f} seconds")
        
        # Channel selection logic
        if channel_selection == 'auto':
            # Priority list for sleep EEG analysis
            preferred_channels = ['Fpz-Cz', 'Pz-Oz', 'C3-A2', 'C4-A1', 'F3-A2', 'F4-A1']
            selected_channel = None
            
            for pref_ch in preferred_channels:
                if pref_ch in channel_names:
                    selected_channel = pref_ch
                    break
            
            if selected_channel is None:
                selected_channel = channel_names[0]  # Fallback to first channel
                
        elif channel_selection == 'first':
            selected_channel = channel_names[0]
        elif isinstance(channel_selection, list):
            selected_channel = channel_selection[0] if channel_selection[0] in channel_names else channel_names[0]
        else:
            selected_channel = channel_names[0]
        
        print(f"  • Selected channel: {selected_channel}")
        
        # Extract data from selected channel
        channel_idx = channel_names.index(selected_channel)
        eeg_data = raw.get_data()[channel_idx]
        
        # Create metadata dictionary
        metadata = {
            'file_path': file_path,
            'selected_channel': selected_channel,
            'sampling_freq': sfreq,
            'n_samples': len(eeg_data),
            'duration_sec': len(eeg_data) / sfreq,
            'all_channels': channel_names
        }
        
        print(f"✅ EEG data loaded successfully")
        print(f"  • Data shape: {eeg_data.shape}")
        print(f"  • Data range: [{eeg_data.min():.2f}, {eeg_data.max():.2f}]")
        
        return eeg_data, sfreq, selected_channel, metadata
        
    except FileNotFoundError:
        print(f"❌ File not found: {file_path}")
        return None, None, None, None
    except Exception as e:
        print(f"❌ Error loading EDF file: {e}")
        return None, None, None, None

# Load EEG data with multiple fallback paths
possible_paths = [
    'by captain borat/raw/EEG_0_per_hour_2024-03-20 17_12_18.edf',
    'raw data/EEG_0_per_hour_2024-03-20 17_12_18.edf',
    'by captain borat/raw/SC4001E0-PSG.edf'
]

eeg_signal = None
for path in possible_paths:
    result = load_eeg_data(path, channel_selection='auto')
    if result[0] is not None:
        eeg_signal, sampling_freq, selected_channel, eeg_metadata = result
        break

if eeg_signal is None:
    print("❌ No valid EEG file found. Please check file paths.")
else:
    print(f"\n📊 EEG Data Summary:")
    print(f"  • Signal length: {len(eeg_signal):,} samples")
    print(f"  • Duration: {len(eeg_signal)/sampling_freq:.1f} seconds ({len(eeg_signal)/sampling_freq/3600:.2f} hours)")
    print(f"  • Mean amplitude: {np.mean(eeg_signal):.2f}")
    print(f"  • Std amplitude: {np.std(eeg_signal):.2f}")

Extracting EDF parameters from /home/yahia/notebooks/by captain borat/raw/EEG_0_per_hour_2024-03-20 17_12_18.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


Reading 0 ... 44153855  =      0.000 ... 86237.998 secs...


## 🔧 Step 2: Signal Preprocessing Pipeline

Comprehensive EEG signal preprocessing including artifact removal, interpolation, and standardization.

In [ ]:
def interpolate_artifacts(signal, sampling_freq, window_duration=10, threshold_factor=3):
    """
    Interpolate artifacts in EEG signal using sliding window approach.
    
    Args:
        signal (np.array): Input EEG signal
        sampling_freq (float): Sampling frequency in Hz
        window_duration (float): Window duration in seconds
        threshold_factor (float): Threshold multiplier for outlier detection
    
    Returns:
        np.array: Interpolated signal with artifacts removed
    """
    
    print(f"🔧 Interpolating artifacts...")
    print(f"  • Window duration: {window_duration}s")
    print(f"  • Threshold factor: {threshold_factor}x std")
    
    interpolated_signal = signal.copy()
    window_size = int(sampling_freq * window_duration)
    
    # Calculate global statistics for reference
    global_std = np.std(signal)
    threshold = threshold_factor * global_std
    
    total_artifacts = 0
    n_windows = 0
    
    # Process signal in sliding windows
    for i in range(0, len(signal), window_size):
        window_end = min(i + window_size, len(signal))
        window = signal[i:window_end]
        
        if len(window) == 0:
            continue
            
        n_windows += 1
        window_mean = np.mean(window)
        
        # Detect outliers (artifacts)
        outliers_mask = np.abs(window - window_mean) > threshold
        n_outliers = np.sum(outliers_mask)
        
        if n_outliers > 0:
            total_artifacts += n_outliers
            # Replace outliers with window mean (simple interpolation)
            interpolated_signal[i:window_end][outliers_mask] = window_mean
    
    artifact_percentage = (total_artifacts / len(signal)) * 100
    
    print(f"  • Processed {n_windows} windows")
    print(f"  • Artifacts detected: {total_artifacts:,} samples ({artifact_percentage:.2f}%)")
    print(f"✅ Artifact interpolation completed")
    
    return interpolated_signal

def standardize_signal(signal):
    """
    Standardize signal to zero mean and unit variance.
    
    Args:
        signal (np.array): Input signal
    
    Returns:
        np.array: Standardized signal
    """
    
    print(f"📊 Standardizing signal...")
    
    mean_val = np.mean(signal)
    std_val = np.std(signal)
    
    if std_val == 0:
        print("⚠️  Warning: Signal has zero variance!")
        return signal - mean_val
    
    standardized = (signal - mean_val) / std_val
    
    print(f"  • Original mean: {mean_val:.4f}, std: {std_val:.4f}")
    print(f"  • Standardized mean: {np.mean(standardized):.4f}, std: {np.std(standardized):.4f}")
    print(f"✅ Signal standardization completed")
    
    return standardized

def remove_linear_trends(signal):
    """
    Remove linear trends from the signal using scipy detrend.
    
    Args:
        signal (np.array): Input signal
    
    Returns:
        np.array: Detrended signal
    """
    
    print(f"📈 Removing linear trends...")
    
    # Calculate trend before removal
    x = np.arange(len(signal))
    slope, intercept = np.polyfit(x, signal, 1)
    
    # Remove trend
    detrended = detrend(signal)
    
    print(f"  • Original trend slope: {slope:.6f}")
    print(f"  • Trend intercept: {intercept:.4f}")
    print(f"✅ Linear trend removal completed")
    
    return detrended

# Apply preprocessing pipeline
if eeg_signal is not None:
    print("🔄 Starting EEG preprocessing pipeline...")
    print("=" * 50)
    
    # Step 1: Interpolate artifacts
    eeg_interpolated = interpolate_artifacts(
        eeg_signal, 
        sampling_freq, 
        window_duration=10, 
        threshold_factor=3
    )
    
    print()
    
    # Step 2: Standardize signal
    eeg_standardized = standardize_signal(eeg_interpolated)
    
    print()
    
    # Step 3: Remove linear trends
    eeg_detrended = remove_linear_trends(eeg_standardized)
    
    print(f"\n📊 Preprocessing Summary:")
    print(f"  • Original signal range: [{eeg_signal.min():.2f}, {eeg_signal.max():.2f}]")
    print(f"  • Final processed range: [{eeg_detrended.min():.2f}, {eeg_detrended.max():.2f}]")
    print(f"  • Processing preserved {len(eeg_detrended):,} samples")
    
else:
    print("❌ No EEG signal available for preprocessing")

## 🤖 Step 3: GPU-Accelerated Hidden Markov Model Implementation

Custom TensorFlow-based HMM implementation with GPU acceleration for large-scale EEG analysis.

In [ ]:
class TensorFlowHMM:
    """
    GPU-accelerated Hidden Markov Model implementation using TensorFlow.
    
    This implementation provides:
    - GPU acceleration for large datasets
    - Expectation-Maximization (EM) algorithm
    - Gaussian emission distributions
    - Viterbi decoding for state prediction
    """
    
    def __init__(self, n_components=5, n_iter=100, tol=1e-4, verbose=True):
        """
        Initialize GPU-accelerated HMM.
        
        Args:
            n_components (int): Number of hidden states
            n_iter (int): Maximum number of EM iterations
            tol (float): Convergence tolerance
            verbose (bool): Print training progress
        """
        self.n_components = n_components
        self.n_iter = n_iter
        self.tol = tol
        self.verbose = verbose
        
        # Model parameters (will be initialized during fit)
        self.means_ = None
        self.covars_ = None
        self.transmat_ = None
        self.startprob_ = None
        self.converged_ = False
        self.n_features_ = None

print("🤖 GPU-accelerated HMM class initialized")

## 🧮 Step 4: HMM Training Methods

Implementation of core HMM algorithms including parameter initialization, EM training, and Viterbi decoding.

In [ ]:
def add_hmm_methods_to_class():
    """Add core HMM methods to the TensorFlowHMM class."""
    
    def _initialize_parameters(self, X):
        """Initialize HMM parameters randomly."""
        n_samples, n_features = X.shape
        self.n_features_ = n_features
        
        # Initialize means using K-means-like approach
        indices = tf.random.uniform([self.n_components], 0, n_samples, dtype=tf.int32)
        self.means_ = tf.Variable(tf.gather(X, indices), dtype=tf.float32)
        
        # Initialize covariances
        self.covars_ = tf.Variable(tf.ones([self.n_components, n_features]), dtype=tf.float32)
        
        # Initialize transition matrix (normalized)
        transmat_raw = tf.random.uniform([self.n_components, self.n_components])
        self.transmat_ = tf.Variable(tf.nn.softmax(transmat_raw, axis=1))
        
        # Initialize start probabilities (normalized)
        startprob_raw = tf.random.uniform([self.n_components])
        self.startprob_ = tf.Variable(tf.nn.softmax(startprob_raw))
    
    def _compute_log_likelihood(self, X):
        """Compute log-likelihood of observations given current parameters."""
        # Expand dimensions for broadcasting
        X_expanded = X[tf.newaxis, :, :]  # [1, n_samples, n_features]
        means_expanded = self.means_[:, tf.newaxis, :]  # [n_components, 1, n_features]
        covars_expanded = self.covars_[:, tf.newaxis, :]  # [n_components, 1, n_features]
        
        # Compute squared Mahalanobis distance
        diff = X_expanded - means_expanded
        mahal_dist = tf.reduce_sum(diff**2 / covars_expanded, axis=2)
        
        # Compute log-likelihood (Gaussian)
        log_likelihood = -0.5 * mahal_dist
        log_likelihood -= 0.5 * tf.reduce_sum(tf.math.log(2 * np.pi * covars_expanded), axis=2)
        
        return log_likelihood
    
    def fit(self, X):
        """Fit HMM using Expectation-Maximization algorithm."""
        X_tf = tf.convert_to_tensor(X, dtype=tf.float32)
        n_samples, n_features = X_tf.shape
        
        if self.verbose:
            print(f"🎯 Training HMM with {self.n_components} states on {n_samples:,} samples...")
        
        # Initialize parameters
        self._initialize_parameters(X_tf)
        
        prev_log_likelihood = -np.inf
        
        for iteration in range(self.n_iter):
            # E-step: Compute forward-backward probabilities
            log_likelihood_matrix = self._compute_log_likelihood(X_tf)
            
            # Simple responsibility calculation (could be improved with forward-backward)
            log_resp = log_likelihood_matrix + tf.math.log(self.startprob_[:, tf.newaxis])
            log_resp = log_resp - tf.reduce_logsumexp(log_resp, axis=0, keepdims=True)
            resp = tf.exp(log_resp)
            
            # M-step: Update parameters
            # Update means
            resp_sum = tf.reduce_sum(resp, axis=1, keepdims=True)
            new_means = tf.matmul(resp, X_tf) / resp_sum
            self.means_.assign(new_means)
            
            # Update covariances
            diff = X_tf[tf.newaxis, :, :] - self.means_[:, tf.newaxis, :]
            weighted_sq_diff = resp[:, :, tf.newaxis] * (diff ** 2)
            new_covars = tf.reduce_sum(weighted_sq_diff, axis=1) / resp_sum
            # Add small regularization to prevent singular covariance
            new_covars = new_covars + 1e-6
            self.covars_.assign(new_covars)
            
            # Compute current log-likelihood for convergence check
            current_log_likelihood = tf.reduce_sum(tf.reduce_logsumexp(log_resp, axis=0))
            
            if self.verbose and (iteration + 1) % 20 == 0:
                print(f"  Iteration {iteration + 1:3d}: Log-likelihood = {current_log_likelihood:.4f}")
            
            # Check convergence
            if abs(current_log_likelihood - prev_log_likelihood) < self.tol:
                if self.verbose:
                    print(f"✅ Converged after {iteration + 1} iterations")
                self.converged_ = True
                break
                
            prev_log_likelihood = current_log_likelihood
        
        if not self.converged_ and self.verbose:
            print(f"⚠️  Did not converge after {self.n_iter} iterations")
        
        return self
    
    def predict(self, X):
        """Predict hidden states using Viterbi algorithm (simplified)."""
        X_tf = tf.convert_to_tensor(X, dtype=tf.float32)
        log_likelihood = self._compute_log_likelihood(X_tf)
        
        # Simple maximum likelihood decoding (could be improved with Viterbi)
        states = tf.argmax(log_likelihood, axis=0)
        return states.numpy()
    
    def predict_proba(self, X):
        """Predict state probabilities for each observation."""
        X_tf = tf.convert_to_tensor(X, dtype=tf.float32)
        log_likelihood = self._compute_log_likelihood(X_tf)
        
        # Convert to probabilities
        log_probs = log_likelihood - tf.reduce_logsumexp(log_likelihood, axis=0, keepdims=True)
        probs = tf.exp(log_probs)
        
        return probs.numpy().T  # Transpose to [n_samples, n_components]
    
    # Add methods to the class
    TensorFlowHMM._initialize_parameters = _initialize_parameters
    TensorFlowHMM._compute_log_likelihood = _compute_log_likelihood
    TensorFlowHMM.fit = fit
    TensorFlowHMM.predict = predict
    TensorFlowHMM.predict_proba = predict_proba

# Add methods to the class
add_hmm_methods_to_class()
print("✅ HMM training methods added to TensorFlowHMM class")

## 🚀 Step 5: HMM Training and State Classification

Apply the GPU-accelerated HMM to classify sleep states from preprocessed EEG signals.

In [ ]:
def prepare_features_for_hmm(signal, feature_window=512, overlap=0.5):
    """
    Prepare features from EEG signal for HMM training.
    
    Args:
        signal (np.array): Preprocessed EEG signal
        feature_window (int): Window size for feature extraction
        overlap (float): Overlap between windows (0-1)
    
    Returns:
        np.array: Feature matrix [n_windows, n_features]
    """
    
    print(f"🔧 Preparing features for HMM...")
    
    step_size = int(feature_window * (1 - overlap))
    n_windows = (len(signal) - feature_window) // step_size + 1
    
    features = []
    
    for i in range(n_windows):
        start_idx = i * step_size
        end_idx = start_idx + feature_window
        
        if end_idx > len(signal):
            break
            
        window = signal[start_idx:end_idx]
        
        # Extract multiple features from each window
        window_features = [
            np.mean(window),           # Mean amplitude
            np.std(window),            # Standard deviation
            np.var(window),            # Variance
            np.max(window),            # Maximum value
            np.min(window),            # Minimum value
            np.percentile(window, 25), # 25th percentile
            np.percentile(window, 75), # 75th percentile
            np.mean(np.abs(window)),   # Mean absolute value
        ]
        
        features.append(window_features)
    
    features_array = np.array(features)
    
    print(f"  • Created {len(features)} feature windows")
    print(f"  • Feature dimension: {features_array.shape[1]}")
    print(f"  • Window size: {feature_window} samples")
    print(f"  • Overlap: {overlap*100:.0f}%")
    
    return features_array

def apply_gpu_hmm_analysis(signal, n_states=5, feature_window=512):
    """
    Apply GPU-accelerated HMM analysis to EEG signal.
    
    Args:
        signal (np.array): Preprocessed EEG signal
        n_states (int): Number of hidden states
        feature_window (int): Window size for feature extraction
    
    Returns:
        dict: HMM analysis results
    """
    
    if signal is None:
        print("❌ No signal provided for HMM analysis")
        return None
    
    print(f"🚀 STARTING GPU-ACCELERATED HMM ANALYSIS")
    print("=" * 50)
    
    # Prepare features
    features = prepare_features_for_hmm(signal, feature_window=feature_window)
    
    # Determine device
    device_name = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'
    print(f"\n🖥️  Using device: {device_name}")
    
    with tf.device(device_name):
        print(f"\n🎯 Training HMM with {n_states} states...")
        
        # Initialize and train HMM
        hmm_model = TensorFlowHMM(
            n_components=n_states,
            n_iter=100,
            tol=1e-4,
            verbose=True
        )
        
        # Fit the model
        start_time = time.time()
        hmm_model.fit(features)
        training_time = time.time() - start_time
        
        print(f"⏱️  Training completed in {training_time:.2f} seconds")
        
        # Predict states
        print(f"\n🔮 Predicting hidden states...")
        predicted_states = hmm_model.predict(features)
        state_probabilities = hmm_model.predict_proba(features)
        
        # Calculate state statistics
        unique_states, state_counts = np.unique(predicted_states, return_counts=True)
        state_percentages = (state_counts / len(predicted_states)) * 100
        
        print(f"\n📊 State Distribution:")
        for state, count, percentage in zip(unique_states, state_counts, state_percentages):
            print(f"  • State {state}: {count:,} windows ({percentage:.1f}%)")
        
        # Create results dictionary
        results = {
            'model': hmm_model,
            'predicted_states': predicted_states,
            'state_probabilities': state_probabilities,
            'features': features,
            'training_time': training_time,
            'n_states': n_states,
            'feature_window': feature_window,
            'state_distribution': dict(zip(unique_states, state_counts)),
            'converged': hmm_model.converged_
        }
        
        print(f"✅ HMM analysis completed successfully")
        return results

# Apply HMM analysis if preprocessed signal is available
if 'eeg_detrended' in locals() and eeg_detrended is not None:
    
    # HMM parameters
    n_hidden_states = 5  # Wake, REM, N1, N2, N3 sleep stages
    feature_window_size = int(sampling_freq)  # 1 second windows
    
    print(f"🧠 Applying Hidden Markov Model for sleep state classification...")
    print(f"  • Number of states: {n_hidden_states}")
    print(f"  • Feature window: {feature_window_size} samples ({feature_window_size/sampling_freq:.1f}s)")
    
    # Run GPU-accelerated HMM
    hmm_results = apply_gpu_hmm_analysis(
        eeg_detrended, 
        n_states=n_hidden_states,
        feature_window=feature_window_size
    )
    
    if hmm_results:
        print(f"\n🎯 HMM Analysis Summary:")
        print(f"  • Training time: {hmm_results['training_time']:.2f} seconds")
        print(f"  • Convergence: {'✅ Yes' if hmm_results['converged'] else '❌ No'}")
        print(f"  • Feature windows: {len(hmm_results['predicted_states']):,}")
        print(f"  • Unique states found: {len(hmm_results['state_distribution'])}")
        
else:
    print("❌ No preprocessed EEG signal available for HMM analysis")

Running HMM on: [LogicalDevice(name='/device:CPU:0', device_type='CPU'), LogicalDevice(name='/device:GPU:0', device_type='GPU')]


2025-06-04 15:30:44.166239: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-04 15:30:44.379233: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10446 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:03:00.0, compute capability: 6.1


In [ ]:
# Alternative: CPU-based HMM using scikit-learn's hmmlearn library
def apply_cpu_hmm_comparison(signal, n_states=5, feature_window=512):
    """
    Apply CPU-based HMM using hmmlearn for performance comparison.
    
    Args:
        signal (np.array): Preprocessed EEG signal
        n_states (int): Number of hidden states
        feature_window (int): Window size for feature extraction
    
    Returns:
        dict: CPU HMM results for comparison
    """
    
    print(f"🖥️  RUNNING CPU HMM FOR COMPARISON")
    print("=" * 40)
    
    # Prepare features (same as GPU version)
    features = prepare_features_for_hmm(signal, feature_window=feature_window)
    
    print(f"🎯 Training CPU-based HMM with {n_states} states...")
    
    try:
        # Initialize CPU HMM model
        cpu_model = hmm.GaussianHMM(
            n_components=n_states,
            covariance_type='diag',
            n_iter=100,
            tol=1e-4,
            verbose=False
        )
        
        # Fit the model
        start_time = time.time()
        cpu_model.fit(features)
        cpu_training_time = time.time() - start_time
        
        print(f"⏱️  CPU training completed in {cpu_training_time:.2f} seconds")
        
        # Predict states
        cpu_states = cpu_model.predict(features)
        cpu_probabilities = cpu_model.predict_proba(features)
        
        # Calculate state statistics
        unique_states, state_counts = np.unique(cpu_states, return_counts=True)
        
        print(f"📊 CPU HMM State Distribution:")
        for state, count in zip(unique_states, state_counts):
            percentage = (count / len(cpu_states)) * 100
            print(f"  • State {state}: {count:,} windows ({percentage:.1f}%)")
        
        cpu_results = {
            'model': cpu_model,
            'predicted_states': cpu_states,
            'state_probabilities': cpu_probabilities,
            'training_time': cpu_training_time,
            'n_states': n_states,
            'converged': True  # hmmlearn doesn't expose convergence status directly
        }
        
        print(f"✅ CPU HMM analysis completed")
        return cpu_results
        
    except Exception as e:
        print(f"❌ CPU HMM failed: {e}")
        return None

# Run CPU HMM for comparison if GPU results are available
cpu_hmm_results = None
if 'hmm_results' in locals() and hmm_results is not None:
    
    print(f"\n🔄 Running CPU comparison...")
    cpu_hmm_results = apply_cpu_hmm_comparison(
        eeg_detrended,
        n_states=n_hidden_states,
        feature_window=feature_window_size
    )
    
    # Performance comparison
    if cpu_hmm_results:
        print(f"\n⚡ PERFORMANCE COMPARISON")
        print("=" * 30)
        
        gpu_time = hmm_results['training_time']
        cpu_time = cpu_hmm_results['training_time']
        
        print(f"GPU Training Time: {gpu_time:.2f} seconds")
        print(f"CPU Training Time: {cpu_time:.2f} seconds")
        
        if gpu_time > 0:
            speedup = cpu_time / gpu_time
            print(f"Speedup Factor: {speedup:.2f}x {'🚀' if speedup > 1 else '🐌'}")
        
        # Compare convergence
        print(f"GPU Convergence: {'✅' if hmm_results['converged'] else '❌'}")
        print(f"CPU Convergence: {'✅' if cpu_hmm_results['converged'] else '❌'}")

else:
    print("⚠️  Skipping CPU comparison - no GPU results available")

CuPy is available for GPU acceleration


## 📊 Step 6: Results Visualization and Analysis

Comprehensive visualization of HMM results including state sequences, probabilities, and model parameters.

In [ ]:
def create_hmm_visualization(hmm_results, cpu_results=None, signal=None, sampling_freq=512):
    """
    Create comprehensive visualization of HMM results.
    
    Args:
        hmm_results (dict): GPU HMM results
        cpu_results (dict): CPU HMM results for comparison
        signal (np.array): Original preprocessed signal
        sampling_freq (float): Sampling frequency
    """
    
    if not hmm_results:
        print("❌ No HMM results to visualize")
        return
    
    print("🎨 Creating HMM visualization...")
    
    # Create figure with subplots
    fig, axes = plt.subplots(3, 2, figsize=(16, 12))
    fig.suptitle('Hidden Markov Model Analysis - Sleep EEG Classification', fontsize=16, fontweight='bold')
    
    # Extract results
    gpu_states = hmm_results['predicted_states']
    gpu_probs = hmm_results['state_probabilities']
    
    # Plot 1: Original signal with state overlay (top portion)
    if signal is not None:
        ax1 = axes[0, 0]
        
        # Show first 10,000 samples for visibility
        signal_portion = signal[:10000]
        time_axis = np.arange(len(signal_portion)) / sampling_freq
        
        ax1.plot(time_axis, signal_portion, 'b-', alpha=0.7, linewidth=0.5, label='EEG Signal')
        ax1.set_xlabel('Time (seconds)')
        ax1.set_ylabel('Amplitude (standardized)')
        ax1.set_title('Preprocessed EEG Signal (First 10k samples)')
        ax1.grid(True, alpha=0.3)
        ax1.legend()
    
    # Plot 2: State sequence
    ax2 = axes[0, 1]
    state_time = np.arange(len(gpu_states)) * (hmm_results['feature_window'] / sampling_freq)
    ax2.plot(state_time, gpu_states, 'o-', markersize=2, linewidth=1, color='red')
    ax2.set_xlabel('Time (seconds)')
    ax2.set_ylabel('Hidden State')
    ax2.set_title('Predicted Sleep States (GPU HMM)')
    ax2.grid(True, alpha=0.3)
    ax2.set_yticks(range(hmm_results['n_states']))
    
    # Plot 3: State probabilities heatmap
    ax3 = axes[1, 0]
    im = ax3.imshow(gpu_probs[:500, :].T, aspect='auto', cmap='viridis', origin='lower')
    ax3.set_xlabel('Time Windows (first 500)')
    ax3.set_ylabel('Hidden State')
    ax3.set_title('State Probabilities Heatmap')
    plt.colorbar(im, ax=ax3, label='Probability')
    
    # Plot 4: State distribution
    ax4 = axes[1, 1]
    state_dist = hmm_results['state_distribution']
    states = list(state_dist.keys())
    counts = list(state_dist.values())
    
    bars = ax4.bar(states, counts, color='lightblue', edgecolor='darkblue')
    ax4.set_xlabel('Hidden State')
    ax4.set_ylabel('Number of Windows')
    ax4.set_title('State Distribution')
    ax4.grid(True, alpha=0.3)
    
    # Add percentage labels on bars
    total_windows = sum(counts)
    for bar, count in zip(bars, counts):
        percentage = (count / total_windows) * 100
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height + 0.01*max(counts),
                f'{percentage:.1f}%', ha='center', va='bottom')
    
    # Plot 5: Comparison with CPU (if available)
    ax5 = axes[2, 0]
    if cpu_results:
        cpu_states = cpu_results['predicted_states']
        
        # Compare first 1000 time points
        comparison_length = min(1000, len(gpu_states), len(cpu_states))
        time_comp = np.arange(comparison_length)
        
        ax5.plot(time_comp, gpu_states[:comparison_length], 'r-', alpha=0.7, label='GPU HMM', linewidth=2)
        ax5.plot(time_comp, cpu_states[:comparison_length], 'b--', alpha=0.7, label='CPU HMM', linewidth=2)
        ax5.set_xlabel('Time Windows')
        ax5.set_ylabel('Predicted State')
        ax5.set_title('GPU vs CPU State Predictions')
        ax5.legend()
        ax5.grid(True, alpha=0.3)
        
        # Calculate agreement
        agreement = np.mean(gpu_states[:comparison_length] == cpu_states[:comparison_length])
        ax5.text(0.02, 0.98, f'Agreement: {agreement:.1%}', transform=ax5.transAxes, 
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    else:
        ax5.text(0.5, 0.5, 'No CPU comparison\navailable', ha='center', va='center', 
                transform=ax5.transAxes, fontsize=12)
        ax5.set_title('CPU Comparison (Not Available)')
    
    # Plot 6: Performance metrics
    ax6 = axes[2, 1]
    
    # Performance data
    perf_labels = ['GPU Training', 'CPU Training'] if cpu_results else ['GPU Training']
    perf_times = [hmm_results['training_time']]
    if cpu_results:
        perf_times.append(cpu_results['training_time'])
    
    colors = ['lightcoral', 'lightblue'] if cpu_results else ['lightcoral']
    bars = ax6.bar(perf_labels, perf_times, color=colors)
    ax6.set_ylabel('Training Time (seconds)')
    ax6.set_title('Training Performance Comparison')
    ax6.grid(True, alpha=0.3)
    
    # Add time labels on bars
    for bar, time_val in zip(bars, perf_times):
        height = bar.get_height()
        ax6.text(bar.get_x() + bar.get_width()/2., height + 0.01*max(perf_times),
                f'{time_val:.2f}s', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ HMM visualization completed")

# Create visualization if results are available
if 'hmm_results' in locals() and hmm_results is not None:
    create_hmm_visualization(
        hmm_results, 
        cpu_results=cpu_hmm_results if 'cpu_hmm_results' in locals() else None,
        signal=eeg_detrended if 'eeg_detrended' in locals() else None,
        sampling_freq=sampling_freq if 'sampling_freq' in locals() else 512
    )
else:
    print("⚠️  No HMM results available for visualization")

## 📈 Step 7: Clinical Analysis and State Interpretation

Analyze HMM results in the context of sleep medicine and provide clinical insights.

In [ ]:
def analyze_sleep_architecture(hmm_results, sampling_freq=512):
    """
    Analyze sleep architecture and provide clinical insights from HMM results.
    
    Args:
        hmm_results (dict): HMM analysis results
        sampling_freq (float): EEG sampling frequency
    
    Returns:
        dict: Clinical analysis results
    """
    
    if not hmm_results:
        print("❌ No HMM results available for clinical analysis")
        return None
    
    print("🏥 CLINICAL SLEEP ARCHITECTURE ANALYSIS")
    print("=" * 50)
    
    states = hmm_results['predicted_states']
    probabilities = hmm_results['state_probabilities']
    feature_window = hmm_results['feature_window']
    
    # Calculate temporal parameters
    window_duration_sec = feature_window / sampling_freq
    total_recording_time = len(states) * window_duration_sec
    
    print(f"Recording Summary:")
    print(f"  • Total recording time: {total_recording_time/3600:.2f} hours")
    print(f"  • Analysis windows: {len(states):,}")
    print(f"  • Window duration: {window_duration_sec:.1f} seconds")
    print(f"  • Detected states: {len(set(states))}")
    
    # State duration analysis
    print(f"\n📊 Sleep State Analysis:")
    print("-" * 30)
    
    state_durations = {}
    current_state = states[0]
    current_duration = 1
    state_episodes = {state: [] for state in set(states)}
    
    # Analyze continuous episodes
    for i in range(1, len(states)):
        if states[i] == current_state:
            current_duration += 1
        else:
            # End of episode
            episode_time_min = (current_duration * window_duration_sec) / 60
            state_episodes[current_state].append(episode_time_min)
            
            current_state = states[i]
            current_duration = 1
    
    # Add final episode
    final_episode_time = (current_duration * window_duration_sec) / 60
    state_episodes[current_state].append(final_episode_time)
    
    # Calculate statistics for each state
    clinical_analysis = {}
    
    for state in sorted(set(states)):
        episodes = state_episodes[state]
        total_time_min = sum(episodes)
        percentage = (total_time_min / (total_recording_time / 60)) * 100
        
        if episodes:
            avg_episode = np.mean(episodes)
            median_episode = np.median(episodes)
            max_episode = np.max(episodes)
            n_episodes = len(episodes)
        else:
            avg_episode = median_episode = max_episode = n_episodes = 0
        
        clinical_analysis[state] = {
            'total_time_min': total_time_min,
            'percentage': percentage,
            'n_episodes': n_episodes,
            'avg_episode_min': avg_episode,
            'median_episode_min': median_episode,
            'max_episode_min': max_episode
        }
        
        print(f"State {state}:")
        print(f"  • Total time: {total_time_min:.1f} min ({percentage:.1f}%)")
        print(f"  • Episodes: {n_episodes}")
        print(f"  • Avg episode: {avg_episode:.1f} min")
        print(f"  • Max episode: {max_episode:.1f} min")
        print()
    
    # Sleep fragmentation analysis
    print(f"🔄 Sleep Fragmentation Analysis:")
    print("-" * 35)
    
    # Count state transitions
    transitions = 0
    for i in range(1, len(states)):
        if states[i] != states[i-1]:
            transitions += 1
    
    transitions_per_hour = transitions / (total_recording_time / 3600)
    fragmentation_index = transitions / len(states)  # Normalized fragmentation
    
    print(f"  • Total transitions: {transitions:,}")
    print(f"  • Transitions per hour: {transitions_per_hour:.1f}")
    print(f"  • Fragmentation index: {fragmentation_index:.3f}")
    
    # State stability analysis
    stability_scores = {}
    for state in set(states):
        state_probs = probabilities[states == state, state]
        if len(state_probs) > 0:
            stability_scores[state] = np.mean(state_probs)
        else:
            stability_scores[state] = 0
    
    print(f"\n🎯 State Confidence Analysis:")
    print("-" * 32)
    for state in sorted(stability_scores.keys()):
        confidence = stability_scores[state]
        print(f"  • State {state} confidence: {confidence:.3f}")
    
    # Clinical interpretation
    print(f"\n🩺 Clinical Interpretation:")
    print("-" * 28)
    
    # Identify potential sleep stages based on temporal patterns
    state_interpretations = {}
    
    for state, analysis in clinical_analysis.items():
        avg_episode = analysis['avg_episode_min']
        percentage = analysis['percentage']
        confidence = stability_scores[state]
        
        # Heuristic sleep stage assignment based on episode duration and prevalence
        if avg_episode > 60 and percentage > 20:  # Long, prevalent episodes
            interpretation = "Potential Deep Sleep (N3/SWS)"
        elif avg_episode > 30 and percentage > 15:  # Medium-long episodes
            interpretation = "Potential Light Sleep (N2)"
        elif avg_episode > 10 and percentage > 10:  # Medium episodes
            interpretation = "Potential Stage 1 Sleep (N1)"
        elif avg_episode < 10 and percentage < 15:  # Short, sporadic episodes
            interpretation = "Potential REM or Transition"
        elif percentage > 25:  # High percentage regardless of episode length
            interpretation = "Potential Wake/Alert State"
        else:
            interpretation = "Unclassified Sleep State"
        
        state_interpretations[state] = interpretation
        print(f"  • State {state}: {interpretation}")
        print(f"    - Confidence: {confidence:.2f}, Duration: {avg_episode:.1f}min, Time: {percentage:.1f}%")
    
    # Summary recommendations
    print(f"\n💡 Analysis Summary & Recommendations:")
    print("-" * 42)
    
    if transitions_per_hour > 30:
        print("  • HIGH fragmentation detected - possible sleep disorder")
    elif transitions_per_hour > 15:
        print("  • MODERATE fragmentation - monitor sleep quality")
    else:
        print("  • NORMAL fragmentation levels - stable sleep")
    
    overall_confidence = np.mean(list(stability_scores.values()))
    if overall_confidence > 0.8:
        print("  • HIGH model confidence - reliable state classification")
    elif overall_confidence > 0.6:
        print("  • MODERATE model confidence - reasonable classification")
    else:
        print("  • LOW model confidence - consider parameter adjustment")
    
    print(f"  • Overall model confidence: {overall_confidence:.3f}")
    print(f"  • Recommended follow-up: {'Clinical review advised' if transitions_per_hour > 20 else 'Normal sleep architecture'}")
    
    return {
        'clinical_analysis': clinical_analysis,
        'transitions_per_hour': transitions_per_hour,
        'fragmentation_index': fragmentation_index,
        'stability_scores': stability_scores,
        'state_interpretations': state_interpretations,
        'overall_confidence': overall_confidence
    }

# Perform clinical analysis if HMM results are available
if 'hmm_results' in locals() and hmm_results is not None:
    clinical_results = analyze_sleep_architecture(
        hmm_results, 
        sampling_freq=sampling_freq if 'sampling_freq' in locals() else 512
    )
    
    print(f"\n✅ Clinical analysis completed successfully")
    
else:
    print("⚠️  No HMM results available for clinical analysis")

print(f"\n🎯 HMM ANALYSIS PIPELINE COMPLETED")
print("=" * 40)
print("✅ GPU-accelerated Hidden Markov Model analysis finished")
print("📊 Sleep state classification and clinical insights generated")
print("🏥 Results ready for clinical interpretation and further analysis")